<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 1.5 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

<b> Group name:</b> DeepLearn18 <br>
<b> Students:</b> Anna-Maria Turca, Mohammed Saeed
  
 
The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 


Generate your final report (export as HTML) and upload it on the submission website http://bigfoot-m1.eurecom.fr/teachingsub/login (using your deeplearnXX/password). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed and submitted by May 30th 2018 (23:59:59 CET).

# Introduction

In the previous Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%. Can you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [45]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels
print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [5]:
# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'

In [4]:
#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 1.288414950
Epoch:  02   =====> Loss= 0.732894853
Epoch:  03   =====> Loss= 0.600243323
Epoch:  04   =====> Loss= 0.536592696
Epoch:  05   =====> Loss= 0.497837072
Epoch:  06   =====> Loss= 0.470999019
Epoch:  07   =====> Loss= 0.451422906
Epoch:  08   =====> Loss= 0.435888112
Epoch:  09   =====> Loss= 0.423260373
Epoch:  10   =====> Loss= 0.413190886
Epoch:  11   =====> Loss= 0.404280690
Epoch:  12   =====> Loss= 0.397043538
Epoch:  13   =====> Loss= 0.390234074
Epoch:  14   =====> Loss= 0.384228658
Epoch:  15   =====> Loss= 0.379303667
Epoch:  16   =====> Loss= 0.374647766
Epoch:  17   =====> Loss= 0.370449246
Epoch:  18   =====> Loss= 0.366696936
Epoch:  19   =====> Loss= 0.362979843
Epoch:  20   =====> Loss= 0.359835466
Epoch:  21   =====> Loss= 0.356841121
Epoch:  22   =====> Loss= 0.353920617
Epoch:  23   =====> Loss= 0.351358843
Epoch:  24   =====> Loss= 0.348678422
Epoch:  25   =====> Loss= 0.346180339
Epoch:  26   =====> Loss= 0.344102740
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

You are now more familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png",width="800" height="600" align="center">
<center><span>Figure 1: Lenet-5 </span></center>





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [6]:
# Functions for weigths and bias initilization 
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0., shape=shape)
  return tf.Variable(initial)

<div class="alert alert-success">

For the <b>first layer</b>, we use a stride of 1 for the <b>Convolution</b> layer to get the required output size. <br> And for the <b>MaxPooling</b> layer, we use a stride of 2 and a windows size of 2 to get the desired output.($$(28-F)/S=14 gives a correct solution for S=F=2)
<br>
<br>Same thing applies for the second convolution layer.<br>
(10-F)/S=5 is solved for S=F=2.

</div>

<div class="alert alert-success">

We now write a function to flatten the output of layer 2.
</div>

In [7]:
def flatten(x):
    x_shape=x.get_shape()
    num_features=(x_shape[1]*x_shape[2]*x_shape[3])
    return tf.reshape(x,[-1,num_features])

<div class="alert alert-success">

We define variables to be used for the LeNet architecture.</div>

In [8]:
num_channels=1
filter_size=5
num_filters_1=6
num_filters_2=16

fc_1=120
fc_2=84
num_classes=10

In [9]:

def LeNet5_Model(image):
    
    #reshape
    x_image = tf.reshape(image,[-1,28,28,1])

    
    
    #Layer1
    conv1_w=weight_variable(shape=[filter_size,filter_size,num_channels,num_filters_1])
    conv1_b=bias_variable([num_filters_1])
    conv1=tf.nn.conv2d(x_image,conv1_w,strides=[1,1,1,1],padding="SAME")+conv1_b
    #print(conv1)
    conv1=tf.nn.relu(conv1)
    pool_1=tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="VALID")
    #print(pool_1)
    
    
    
    #Layer2
    conv2_w=weight_variable(shape=[filter_size,filter_size,num_filters_1,num_filters_2])
    conv2_b=bias_variable([num_filters_2])
    conv2=tf.nn.conv2d(pool_1,conv2_w,strides=[1,1,1,1],padding="VALID")+conv2_b
    #print(conv2)
    conv2=tf.nn.relu(conv2)
    pool_2=tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="VALID")
    #print(pool_2)
    
    #Flatten
    flattened=flatten(pool_2)
    #print(flattened)
    
    
    #Layer3
    fc1_w=weight_variable(shape=[400,120])
    fc1_b=bias_variable([120])
    fc1=tf.matmul(flattened,fc1_w)+fc1_b
    
    fc1=tf.nn.relu(fc1)
    
    
    #Layer4
    fc2_w=weight_variable(shape=[120,84])
    fc2_b=bias_variable([84])
    fc2=tf.matmul(fc1,fc2_w)+fc2_b
    
    fc2=tf.nn.relu(fc2)
    
    
    #Layer5
    fc3_w=weight_variable(shape=[84,10])
    fc3_b=bias_variable([10])
    logits=tf.matmul(fc2,fc3_w)+fc3_b
    
    return tf.nn.softmax(logits)
    
    
    
    


<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 


<div class="alert alert-success">
The number of parameters of each layer is the number of elements of the filter multiplied by the number of inputs plus the number of biases.<br>
<strong>Number of parameters of Layer 1</strong>: 5x5x1x6 +6=<strong>156</strong><br>
<strong>Number of parameters of Layer 2</strong>: 5x5x1x16+16=<strong>416</strong><br>
<strong>Number of parameters of Layer 3</strong>: 400x120+120=<strong>48120</strong><br>
<strong>Number of parameters of Layer 4</strong>: 120x84+84=<strong>10164</strong><br>
<strong>Number of parameters of Layer 5</strong>: 84x10+10=<strong>850</strong>
</div>

<div class="alert alert-success">
The total number of parameters is <b>59706</b> parameters.</div>


<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Fucntion: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [9]:
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
learning_rate = 0.001
training_epochs = 40
batch_size = 128
logs_path = 'log_files/'


x = tf.placeholder(tf.float32, [None, 784], name='InputData')
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

with tf.name_scope('Model'):
    pred=LeNet5_Model(x)

with tf.name_scope('Loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))

with tf.name_scope('SGD'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)




<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [10]:
def evaluate(logits, labels):
    # logits will be the outputs of your model, labels will be one-hot vectors corresponding to the actual labels
    # logits and labels are numpy arrays
    # this function should return the accuracy of your model
    
    y_pred_classes=tf.argmax(logits,axis=1)
    y_true_classes=tf.argmax(labels,axis=1)
    
    correct_prediction = tf.equal(y_pred_classes, y_true_classes)
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return accuracy


In [11]:
with tf.name_scope('Accuracy'):
    acc = evaluate(pred,y)

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [12]:
# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy  tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [16]:

training_epochs=40
display_step=1
n=-1
saving_path = 'models/'

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    saver=tf.train.Saver()
    # Training cycle
    start_time=time.time()
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
            
        train_acc=acc.eval(feed_dict={x: mnist.train.images, y: mnist.train.labels})
        val_acc=acc.eval(feed_dict={x: mnist.validation.images,y: mnist.validation.labels})
        test_acc=acc.eval(feed_dict={x: mnist.test.images,y: mnist.test.labels})


        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost),
                  "  Train Accuracy=", "{:.9f}".format(train_acc),
                  "  Validation Accuracy=", "{:.9f}".format(val_acc))
        if (test_acc>0.99):
            n=epoch+1
            break
    end_time=time.time()
    time_taken=end_time-start_time
    print("Training Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("---------------------------------------------------------------------------------------------------------")
    print("Final Accuracy:", test_acc)
    if(n!=-1):
        print("Test Accuracy over 99 percent reached after %d epochs" %(n))
    else:
        print("Test Accuracy did not reach 99 percent.")

    save_path = saver.save(sess, saving_path+"/model"+str(int(time.time())))

    print("Elapsed Time: ",time_taken,"seconds.")

Epoch:  01   =====> Loss= 2.299616704   Train Accuracy= 0.159072727   Validation Accuracy= 0.156800002
Epoch:  02   =====> Loss= 2.281474473   Train Accuracy= 0.231054544   Validation Accuracy= 0.232600003
Epoch:  03   =====> Loss= 2.253267493   Train Accuracy= 0.343909085   Validation Accuracy= 0.341399997
Epoch:  04   =====> Loss= 2.194353607   Train Accuracy= 0.415781826   Validation Accuracy= 0.408800006
Epoch:  05   =====> Loss= 2.051322230   Train Accuracy= 0.488127261   Validation Accuracy= 0.481000006
Epoch:  06   =====> Loss= 1.709595099   Train Accuracy= 0.685872734   Validation Accuracy= 0.686800003
Epoch:  07   =====> Loss= 1.177147973   Train Accuracy= 0.778781831   Validation Accuracy= 0.784200013
Epoch:  08   =====> Loss= 0.781942644   Train Accuracy= 0.829690933   Validation Accuracy= 0.838199973
Epoch:  09   =====> Loss= 0.587153607   Train Accuracy= 0.857127249   Validation Accuracy= 0.865599990
Epoch:  10   =====> Loss= 0.488564186   Train Accuracy= 0.872854531   Val

<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.


<img src="MNIST_figures/Accuracy.png",width="800" height="600" align="center">
<center><span>Figure 2: Accuracy </span></center>





<img src="MNIST_figures/Loss.png",width="800" height="600" align="center">
<center><span>Figure 3: Loss </span></center>



<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer and then fill the table above:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |      95.57%        |        99 .03%      |       
| Training Time        |      27.45 min     |        12.47 min    |  

- Which optimizer gives the best accuracy on test data?

**We can see that the AdamOptimiser gave better results than SGD. We managed to reach an accuracy around 99% on the test set with AdamOptimiser which we didn't reach with SGD and with a much less time. The reason why ADAM is better is that it  calculates an exponential moving average of the gradient and the squared gradient, and the parameters beta1 and beta2 control the decay rates of these moving averages.** 


In [16]:
def train(model,learning_rate=0.001,training_epochs=40,batch_size=128,logs_path='log_files/',saving_path='models/',optimiser="Adam",display_step=1):
    
    tf.reset_default_graph()

    x = tf.placeholder(tf.float32, [None, 784], name='InputData')
    y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

    with tf.name_scope('Model'):
        pred=model(x)

    with tf.name_scope('Loss'):
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
    
    
    if(optimiser=="Adam"):
        with tf.name_scope('AdamOptimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    else:
        with tf.name_scope('SGD'):
            optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)


    with tf.name_scope('Accuracy'):
        acc = evaluate(pred,y)

    # Initializing the variables
    init = tf.global_variables_initializer()
    # Create a summary to monitor cost tensor
    tf.summary.scalar("Loss", cost)
    # Create a summary to monitor accuracy  tensor
    tf.summary.scalar("Accuracy", acc)
    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()    


    n=-1


    # Launch the graph for training
    with tf.Session() as sess:
        sess.run(init)
        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter(logs_path+'ADAM', graph=tf.get_default_graph())
        saver=tf.train.Saver()

        # Training cycle
        start_time=time.time()
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                         feed_dict={x: batch_xs, y: batch_ys})
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch * total_batch + i)
                # Compute average loss
                avg_cost += c / total_batch
                
            train_acc=acc.eval(feed_dict={x: mnist.train.images,y: mnist.train.labels})
            val_acc=acc.eval(feed_dict={x: mnist.validation.images,y: mnist.validation.labels})
            test_acc=acc.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})
            # Display logs per epoch step
            if (epoch+1) % display_step == 0:
                print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost),
                "  Validation Accuracy=", "{:.9f}".format(val_acc), "  Test Accuracy=", "{:.9f}".format(test_acc))
            if (test_acc>0.99):
                n=epoch+1
                break

        end_time=time.time()
        time_taken=end_time-start_time
        print("Training Finished!")
        summary_writer.flush()

        # Test model
        # Calculate accuracy
        print("-------------------------------------------------------------------------------------------")
        if(n!=-1):
            print("Test Accuracy over 99 percent reached after %d epochs" %(n))
        else:
            print("Test Accuracy did not reach 99 percent.")        
        print("Final Accuracy:", test_acc)
        save_path = saver.save(sess, saving_path+"model_adam"+str(int(time.time())))

        print("Elapsed Time: ",time_taken,"seconds.")

In [31]:
train(LeNet5_Model,learning_rate=0.001,training_epochs=40,batch_size=128,logs_path='log_files/',saving_path='models/',optimiser="Adam",display_step=1)

Epoch:  01   =====> Loss= 0.342669716   Validation Accuracy= 0.966799974   Test Accuracy= 0.968200028
Epoch:  02   =====> Loss= 0.088865482   Validation Accuracy= 0.983399987   Test Accuracy= 0.982500017
Epoch:  03   =====> Loss= 0.062367673   Validation Accuracy= 0.984799981   Test Accuracy= 0.985199988
Epoch:  04   =====> Loss= 0.047671252   Validation Accuracy= 0.986400008   Test Accuracy= 0.987299979
Epoch:  05   =====> Loss= 0.038379036   Validation Accuracy= 0.987200022   Test Accuracy= 0.988600016
Epoch:  06   =====> Loss= 0.031156608   Validation Accuracy= 0.986999989   Test Accuracy= 0.988799989
Epoch:  07   =====> Loss= 0.026187367   Validation Accuracy= 0.988399982   Test Accuracy= 0.988399982
Epoch:  08   =====> Loss= 0.022499809   Validation Accuracy= 0.987999976   Test Accuracy= 0.988099992
Epoch:  09   =====> Loss= 0.019960781   Validation Accuracy= 0.987200022   Test Accuracy= 0.987699986
Epoch:  10   =====> Loss= 0.017117410   Validation Accuracy= 0.983799994   Test Ac

<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

**Accuracy achieved on testing data:** 98.88%

In [22]:
def LeNet5_Model_Dropout(image,keep_prob=0.75):    

    #reshape
    x_image = tf.reshape(image,[-1,28,28,1])

    
    
    #Layer1
    conv1_w=weight_variable(shape=[filter_size,filter_size,num_channels,num_filters_1])
    conv1_b=bias_variable([num_filters_1])
    conv1=tf.nn.conv2d(x_image,conv1_w,strides=[1,1,1,1],padding="SAME")+conv1_b
    #print(conv1)
    conv1=tf.nn.relu(conv1)
    pool_1=tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="VALID")
    
    #print(pool_1)
    
    
    
    #Layer2
    conv2_w=weight_variable(shape=[filter_size,filter_size,num_filters_1,num_filters_2])
    conv2_b=bias_variable([num_filters_2])
    conv2=tf.nn.conv2d(pool_1,conv2_w,strides=[1,1,1,1],padding="VALID")+conv2_b
    #print(conv2)
    conv2=tf.nn.relu(conv2)
    pool_2=tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="VALID")
    #print(pool_2)
    
    #Flatten
    flattened=flatten(pool_2)
    #print(flattened)
    
    
    #Layer3
    fc1_w=weight_variable(shape=[400,120])
    fc1_b=bias_variable([120])
    fc1=tf.matmul(flattened,fc1_w)+fc1_b
    
    fc1=tf.nn.relu(fc1)
    fc1_drop=tf.nn.dropout(fc1,keep_prob)
    
    #Layer4
    fc2_w=weight_variable(shape=[120,84])
    fc2_b=bias_variable([84])
    fc2=tf.matmul(fc1_drop,fc2_w)+fc2_b
    
    fc2=tf.nn.relu(fc2)
    
    #fc2_drop=tf.nn.dropout(fc2,keep_prob)
    
    #Layer5
    fc3_w=weight_variable(shape=[84,10])
    fc3_b=bias_variable([10])
    logits=tf.matmul(fc2,fc3_w)+fc3_b
    
    return tf.nn.softmax(logits)


In [11]:
train(LeNet5_Model_Dropout,learning_rate=0.001,training_epochs=40,batch_size=128,logs_path='log_files/ADAM_DROPOUT',saving_path='models/ADAM_DROPOUT',optimiser="Adam",display_step=1)

Epoch:  01   =====> Loss= 0.391572584   Validation Accuracy= 0.963199973   Test Accuracy= 0.960099995
Epoch:  02   =====> Loss= 0.112643845   Validation Accuracy= 0.973800004   Test Accuracy= 0.975099981
Epoch:  03   =====> Loss= 0.083854699   Validation Accuracy= 0.978999972   Test Accuracy= 0.978900015
Epoch:  04   =====> Loss= 0.065895880   Validation Accuracy= 0.980000019   Test Accuracy= 0.982599974
Epoch:  05   =====> Loss= 0.056328771   Validation Accuracy= 0.983399987   Test Accuracy= 0.983200014
Epoch:  06   =====> Loss= 0.048526168   Validation Accuracy= 0.983600020   Test Accuracy= 0.983299971
Epoch:  07   =====> Loss= 0.042504138   Validation Accuracy= 0.985000014   Test Accuracy= 0.984600008
Epoch:  08   =====> Loss= 0.036820045   Validation Accuracy= 0.985000014   Test Accuracy= 0.984600008
Epoch:  09   =====> Loss= 0.035398948   Validation Accuracy= 0.986400008   Test Accuracy= 0.986699998
Epoch:  10   =====> Loss= 0.031483554   Validation Accuracy= 0.986000001   Test Ac


<div class="alert alert-success">
Adding dropout allowed us to reach 99% accuracy on the test data with only 7 minutes compared to 12 minutes without dropout.
</div>


<div class="alert alert-success">
Let's try now adding dropout to other layers.
</div>

In [12]:
def LeNet5_Model_Dropout2(image,keep_prob=0.75):    

    #reshape
    x_image = tf.reshape(image,[-1,28,28,1])

    
    
    #Layer1
    conv1_w=weight_variable(shape=[filter_size,filter_size,num_channels,num_filters_1])
    conv1_b=bias_variable([num_filters_1])
    conv1=tf.nn.conv2d(x_image,conv1_w,strides=[1,1,1,1],padding="SAME")+conv1_b
    #print(conv1)
    conv1=tf.nn.relu(conv1)
    pool_1=tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="VALID")
    
    #print(pool_1)
    
    
    
    #Layer2
    conv2_w=weight_variable(shape=[filter_size,filter_size,num_filters_1,num_filters_2])
    conv2_b=bias_variable([num_filters_2])
    conv2=tf.nn.conv2d(pool_1,conv2_w,strides=[1,1,1,1],padding="VALID")+conv2_b
    #print(conv2)
    conv2=tf.nn.relu(conv2)
    pool_2=tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="VALID")
    #print(pool_2)
    
    #Flatten
    flattened=flatten(pool_2)
    #print(flattened)
    
    
    #Layer3
    fc1_w=weight_variable(shape=[400,120])
    fc1_b=bias_variable([120])
    fc1=tf.matmul(flattened,fc1_w)+fc1_b
    
    fc1=tf.nn.relu(fc1)
    fc1_drop=tf.nn.dropout(fc1,keep_prob)
    
    #Layer4
    fc2_w=weight_variable(shape=[120,84])
    fc2_b=bias_variable([84])
    fc2=tf.matmul(fc1_drop,fc2_w)+fc2_b
    
    fc2=tf.nn.relu(fc2)
    
    fc2_drop=tf.nn.dropout(fc2,keep_prob)
    
    #Layer5
    fc3_w=weight_variable(shape=[84,10])
    fc3_b=bias_variable([10])
    logits=tf.matmul(fc2_drop,fc3_w)+fc3_b
    
    return tf.nn.softmax(logits)


In [17]:
train(LeNet5_Model_Dropout2,learning_rate=0.001,training_epochs=40,batch_size=128,logs_path='log_files/ADAM_DROPOUT2',saving_path='models/ADAM_DROPOUT2',optimiser="Adam",display_step=1)

Epoch:  01   =====> Loss= 0.513664424   Validation Accuracy= 0.953199983   Test Accuracy= 0.944899976
Epoch:  02   =====> Loss= 0.149366778   Validation Accuracy= 0.967400014   Test Accuracy= 0.965300024
Epoch:  03   =====> Loss= 0.105040194   Validation Accuracy= 0.973999977   Test Accuracy= 0.974699974
Epoch:  04   =====> Loss= 0.082503379   Validation Accuracy= 0.979600012   Test Accuracy= 0.978299975
Epoch:  05   =====> Loss= 0.071435529   Validation Accuracy= 0.983399987   Test Accuracy= 0.981700003
Epoch:  06   =====> Loss= 0.060406025   Validation Accuracy= 0.980799973   Test Accuracy= 0.981599987
Epoch:  07   =====> Loss= 0.055404045   Validation Accuracy= 0.983600020   Test Accuracy= 0.981700003
Epoch:  08   =====> Loss= 0.047677740   Validation Accuracy= 0.984000027   Test Accuracy= 0.981199980
Epoch:  09   =====> Loss= 0.045840917   Validation Accuracy= 0.981400013   Test Accuracy= 0.980700016
Epoch:  10   =====> Loss= 0.039982803   Validation Accuracy= 0.982999980   Test Ac


<div class="alert alert-success">
Adding dropout to other layers didn't yield us an accuracy of 99% on the test data.
</div>


<div class="alert alert-success">
Let's try exponentially decreasing the learning rate.
</div>

In [42]:
def train(model,training_epochs=40,batch_size=128,logs_path='log_files/',saving_path='models/',display_step=1):
    
    tf.reset_default_graph()

    x = tf.placeholder(tf.float32, [None, 784], name='InputData')
    y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

    with tf.name_scope('Model'):
        pred=model(x)

    with tf.name_scope('Loss'):
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
    
    
        with tf.name_scope('AdamOptimizer'):
            
            batch = tf.Variable(0)
            
            train_size=len(X_train)
            learning_rate = tf.train.exponential_decay(
            1e-4,  # Base learning rate.
            batch * batch_size,  # Current index into the dataset.
            train_size,  # Decay step.
            0.95,  # Decay rate.
            staircase=True)
            optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
            
    
    
    
    with tf.name_scope('Accuracy'):
        acc = evaluate(pred,y)

    # Initializing the variables
    init = tf.global_variables_initializer()
    # Create a summary to monitor cost tensor
    tf.summary.scalar("Loss", cost)
    # Create a summary to monitor accuracy  tensor
    tf.summary.scalar("Accuracy", acc)
    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()    




    # Launch the graph for training
    with tf.Session() as sess:
        sess.run(init)
        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter(logs_path+'ADAM', graph=tf.get_default_graph())
        saver=tf.train.Saver()

        # Training cycle
        start_time=time.time()
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                         feed_dict={x: batch_xs, y: batch_ys})
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch * total_batch + i)
                # Compute average loss
                avg_cost += c / total_batch
                
            train_acc=acc.eval(feed_dict={x: mnist.train.images,y: mnist.train.labels})
            val_acc=acc.eval(feed_dict={x: mnist.validation.images,y: mnist.validation.labels})
            test_acc=acc.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels})
            # Display logs per epoch step
            if (epoch+1) % display_step == 0:
                print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost),
                "  Validation Accuracy=", "{:.9f}".format(val_acc), "  Test Accuracy=", "{:.9f}".format(test_acc))


        end_time=time.time()
        time_taken=end_time-start_time
        print("Training Finished!")
        summary_writer.flush()

        # Test model
        # Calculate accuracy
        print("-------------------------------------------------------------------------------------------")      
        print("Final Accuracy:", test_acc)
        save_path = saver.save(sess, saving_path+"model_adam"+str(int(time.time())))

        print("Elapsed Time: ",time_taken,"seconds.")

In [44]:
train(LeNet5_Model_Dropout,training_epochs=40,batch_size=128,logs_path='log_files/LeNet2',saving_path='models/LeNet2',display_step=1)

Epoch:  01   =====> Loss= 1.396593710   Validation Accuracy= 0.821600020   Test Accuracy= 0.819599986
Epoch:  02   =====> Loss= 0.446355302   Validation Accuracy= 0.898800015   Test Accuracy= 0.902000010
Epoch:  03   =====> Loss= 0.299992197   Validation Accuracy= 0.922800004   Test Accuracy= 0.926299989
Epoch:  04   =====> Loss= 0.234137837   Validation Accuracy= 0.934199989   Test Accuracy= 0.938199997
Epoch:  05   =====> Loss= 0.198031092   Validation Accuracy= 0.949400008   Test Accuracy= 0.948300004
Epoch:  06   =====> Loss= 0.172616107   Validation Accuracy= 0.953199983   Test Accuracy= 0.953700006
Epoch:  07   =====> Loss= 0.152028604   Validation Accuracy= 0.962199986   Test Accuracy= 0.957899988
Epoch:  08   =====> Loss= 0.137433209   Validation Accuracy= 0.959999979   Test Accuracy= 0.959800005
Epoch:  09   =====> Loss= 0.126874997   Validation Accuracy= 0.964600027   Test Accuracy= 0.964999974
Epoch:  10   =====> Loss= 0.117239229   Validation Accuracy= 0.971400023   Test Ac


<div class="alert alert-success">
We didn't get the 99% accuracy we wanted. Other things we could do is tune the decay factor and apply batch normalisation the layers.</div>